<a href="https://colab.research.google.com/github/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/recipe_obesity_pkl_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing Dependencies

In [30]:
import numpy as np
import pandas as pd

In [31]:
!git clone https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship.git
%cd Feynn-Labs-Machine-Learning-Internship

Cloning into 'Feynn-Labs-Machine-Learning-Internship'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 296 (delta 88), reused 5 (delta 5), pack-reused 127 (from 1)
Receiving objects: 100% (296/296), 11.26 MiB | 9.94 MiB/s, done.
Resolving deltas: 100% (128/128), done.
/content/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship


##Load the trained models and the recipe dataset

In [32]:
# Load the Random Forest model
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

# Load the label encoder
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load the feature scaler
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
# Load the trained recipe prediction model from the pickle file
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/trained_diab_recep_model.pkl', 'rb') as file:
    loaded_models = pickle.load(file)
# Load the dataset
data = pd.read_csv('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/filtered_datasets/diab_recipe_df.csv')

##Performing the predictions based on random input data on the Obesity prediction model

In [33]:
# Define the new data point
new_data = {
    'Gender': [1],  # Male (0 for Female, 1 for Male)
    'Age': [25],
    'Height': [1.75],
    'Weight': [30],
    'family_history_with_overweight': [1],  # Yes (1) or No (0)
    'FCVC': [2],  # Food Consumption per Day
    'NCP': [3],  # Number of Main Meals
    'CAEC': [1],  # Always (1) or Sometimes (0) for 'Consumption of Food between Meals'
    'CH2O': [2],  # Daily Water Intake
    'FAF': [2],  # Physical Activity Frequency
    'TUE': [2],  # Time using technology
    'CALC': [0]   # Alcohol Consumption (No)
}

# Convert the new data into a DataFrame
new_data_df = pd.DataFrame(new_data)

# Scale the new data using the previously saved scaler
new_data_scaled = scaler.transform(new_data_df)

# Predict the obesity type using the loaded Random Forest model
prediction = rf_model.predict(new_data_scaled)



In [34]:
# Decode the prediction (if necessary)
decoded_prediction = label_encoder.inverse_transform(prediction)

print(f"Predicted Obesity Type: {decoded_prediction[0]}")

Predicted Obesity Type: Normal_Weight


##Performing the predictions based on predicted output i.e., obesity type as the input data for the recipe prediction model.

In [35]:
# Bug handling at the occurance of "Normal Weight"
if decoded_prediction[0] == 'Normal_Weight':
  decoded_prediction[0] = decoded_prediction[0].replace("_", " ")


# Example user input (current obesity type)
user_obesity_type = decoded_prediction[0]

# Prepare recommendations based on user obesity type
def recommend_recipes(user_obesity_type, data, models):
    recommendations = []

    for index, row in data.iterrows():
        new_recipe = pd.DataFrame({
            'Calories': [row['Calories']],
            'Sugar (g)': [row['Sugar (g)']],
            'Total Fat (g)': [row['Total Fat (g)']],
            'Saturated Fat (g)': [row['Saturated Fat (g)']]
        })

        # Predict for the user’s obesity type
        prediction = models[user_obesity_type].predict(new_recipe)

        if prediction[0] == 1:
            recommendations.append({
                'Recipe Name': row['Recipe Name'],
                'Cuisine': row['Cuisine'],  # Include cuisine information
                'Calories': row['Calories'],
                'Sugar (g)': row['Sugar (g)'],
                'Total Fat (g)': row['Total Fat (g)'],
                'Saturated Fat (g)': row['Saturated Fat (g)']
            })

    # Create DataFrame from recommendations
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

# Get recommendations for the user using the loaded models
recommended_recipes_df = recommend_recipes(user_obesity_type, data, loaded_models)

# Display the DataFrame with recommended recipes, their nutritional details, and cuisine
print(f"Recommended recipes for {user_obesity_type}:")
recommended_recipes_df


Recommended recipes for Normal Weight:


,Recipe Name,Cuisine,Calories,Sugar (g),Total Fat (g),Saturated Fat (g)
0,Punugulu,Andhra Pradesh,1057.899202,13.100185,3.945037,0.767981
1,Andhra Gunpowder,Andhra Pradesh,134.521833,4.477519,5.776929,3.226080
2,Ginger Chutney,Andhra Pradesh,75.018000,7.984223,0.785400,0.163840
3,Goan-style vegetable curry with kitchari,Goan,2243.694000,9.402000,16.154400,2.569170
4,Goan Dal recipes,Goan,651.748075,13.691802,3.116443,0.457488
...,...,...,...,...,...,...
61,"Panch Phoron, Bengali Five-Spice",Bengali,54.076000,0.192050,3.218470,0.344560
62,Panchphoron – The Bengali Five Spice Blend…,Bengali,323.874000,1.179540,19.334620,2.068140
63,"Chirer Pulao, Bengali style",Bengali,1158.394625,18.749750,20.598096,2.002731
64,Spicy Cauliflower Curry Bengali Style |Dho,Bengali,312.090500,11.424547,19.484725,2.129815


Based on the suggested nutritional details, many of the recommended recipes seem to be suitable for individuals with Obesity Type III, particularly due to their low sugar and fat content. However, calorie content varies, so the total daily intake should be monitored to ensure it aligns with the individual's dietary needs.

Personalization: It's always best to consult a healthcare provider or a dietitian to tailor dietary recommendations to individual health needs and goals. They can provide more personalized advice based on a complete assessment of health status and dietary requirements.